In [1]:
#analiza regresyjna za pomocą konwolucyjnej sieci neuronowej
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import datetime
import os

In [2]:
def create_regression_cnn(input_shape=(64,64,3)):
    model = models.Sequential([
        #wejście
        layers.Input(shape=input_shape),
        layers.Rescaling(1./255),

        #trzy bloki konwolucyjne
        layers.Conv2D(32, (3,3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(64, (3,3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(128, (3,3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2,2)),

        #warstwy gęste
        layers.Flatten(),
        layers.Dense(256,activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(64,activation='relu'),
        layers.Dense(1) #regresja - pojedyncza wartosc

    ])
    return model

In [3]:
import numpy as np

#przykładowe dane
x_train = np.random.rand(1000,64,64,3)
y_train = np.random.rand(1000,1)
x_val = np.random.rand(200,64,64,3)
y_val = np.random.rand(200,1)
x_test = np.random.rand(1000,64,64,3)
y_test = np.random.rand(1000,1)

In [4]:
#definicja callbacks
log_dir = os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d - %H%M%S"))
tensorboard_cb = callbacks.TensorBoard(log_dir=log_dir)
earlystop_cb = callbacks.EarlyStopping(patience=5,restore_best_weights=True)
checkpoint_cb = callbacks.ModelCheckpoint("best_model.h5",save_best_only=True)

In [5]:
model = create_regression_cnn()
model.compile(optimizer='adam',loss='mse',metrics=['mae'])

In [6]:
model.fit(x_train,y_train,
          validation_data=(x_val,y_val),
          epochs=50,
          batch_size =32,
          callbacks=[earlystop_cb,checkpoint_cb,tensorboard_cb])

Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - loss: 17.8905 - mae: 3.0141

32/32 ━━━━━━━━━━━━━━━━━━━━ 13s 160ms/step - loss: 17.6519 - mae: 2.9902 - val_loss: 0.3082 - val_mae: 0.4753
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.7418 - mae: 1.0391

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 1.7264 - mae: 1.0337 - val_loss: 0.2429 - val_mae: 0.4060
Epoch 3/50
29/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5178 - mae: 0.5528

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.5109 - mae: 0.5494 - val_loss: 0.1922 - val_mae: 0.3522
Epoch 4/50
28/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2816 - mae: 0.4205

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.2797 - mae: 0.4191 - val_loss: 0.1431 - val_mae: 0.3028
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1603 - mae: 0.3167

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.1604 - mae: 0.3170 - val_loss: 0.1094 - val_mae: 0.2707
Epoch 6/50
26/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1378 - mae: 0.2979

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.1364 - mae: 0.2973 - val_loss: 0.0831 - val_mae: 0.2496
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.1150 - mae: 0.2793 - val_loss: 0.0980 - val_mae: 0.2676
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.1089 - mae: 0.2734 - val_loss: 0.1518 - val_mae: 0.3247
Epoch 9/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1082 - mae: 0.2793 - val_loss: 0.2271 - val_mae: 0.3988
Epoch 10/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.1036 - mae: 0.2684 - val_loss: 0.3202 - val_mae: 0.4894
Epoch 11/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0931 - mae: 0.2621 - val_loss: 0.3827 - val_mae: 0.5480


In [7]:
loss,mae = model.evaluate(x_test,y_test)
print(f"Test mAE: {mae:.4f}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0855 - mae: 0.2481
Test mAE: 0.2475
